# Отчет о решении задачи

Задача сформулирована так - даны объекты вида (номер товара, год продаж, неделя продаж, 64 признака без имен), надо определить величину продаж через shift недель.

Задача оказалась уязвима - можно использовать признаки тестовой выборки для получения ответа на ней же. Поскольку контест задумывался не об этом, делать я этого не стал.

Было испробовано много моделей из разных библиотек, была попытка бинаризовать item_id, попытка изменить таргеты, а потом выполнить обратное преобразование, попытка поменять параметры. Все это работает хуже, чем простое финальное решение. Лучше всего из неудачных попыток сработал XGBoost на функции потерь $(x-y)^{1.5}$ с высотой деревьев 15 (т.к. SMAPE похож на MAE, но MAE использовать не вышло, т.к. функция должна быть выпуклой).

В итоге выяснилось, что тестовые данные - это предсказания для первых 10 недель 2016 года, а обучающие данные - предсказания с 2012 до начала 2016. Итоговое решение разбивает объекты на группы по item_id и для каждого item'а берет последнее известное значение. Чтобы избежать переобучение на public leaderboard, для всех item'ов посчитана XGBoost-модель на них. Если кросс-валидация показывала хороший результат и значение не сильно отличалось от последнего, оно заменялось предсказанием на модели. Код решения предоставлен ниже.

Решения кросс-валидировались на TimeSeriesSplit, т.к. данные привязаны к времени. Результаты отличались от public leaderboard на не более 4% по SMAPE.

# Код решения

In [1]:
%pylab inline
import pandas as pd
from sklearn.model_selection import TimeSeriesSplit

Populating the interactive namespace from numpy and matplotlib


In [2]:
train = pd.read_csv("train.tsv")
test = pd.read_csv("test.tsv")

X_all = np.array(train.drop(['Num', 'y', 'item_id'], axis=1))
X_test = np.array(test.drop(['Num', 'item_id'], axis=1))
y_all = np.array(train['y'])

In [3]:
X = {}
y = {}
last = {}
for i in range(len(train['y'])):
    item = train['item_id'][i]
    if item not in X:
        X[item] = []
        y[item] = []
        last[item] = []
    X[item].append(X_all[i])
    y[item].append(y_all[i])
    last[item].append(y_all[i])


In [4]:
def smape(y1, y2):
    return 200 * (np.abs(y1 - y2) / (np.abs(y1) + np.abs(y2))).sum() / len(y1)

def cv(model, X, y):
    tscv = TimeSeriesSplit(n_splits=3)
    scores = []
    for train, test in tscv.split(X):
        model.fit(X[train], y[train])
        y_pred = model.predict(X[test])
        
        #print("test %.2f, train %.2f" % (smape(y_pred, y[test]), smape(model.predict(X[train]), y[train])))
        scores.append(smape(y_pred, y[test]))
    print("smape %.2f" % np.mean(scores))
    return np.mean(scores)

In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import LinearSVR
from xgboost import XGBRegressor
from sklearn.neighbors import KNeighborsRegressor
model = {}
for item in X.keys():
    model[item] = XGBRegressor()
    k = len(X[item])
    if k < 300 or cv(model[item], np.array(X[item]), np.array(y[item])) > 25:
        model[item] = None
    else:
        model[item].fit(X[item], y[item])

smape 12.65
smape 29.29
smape 59.22
smape 43.15
smape 50.86
smape 74.82
smape 65.46
smape 25.30
smape 52.95
smape 20.10
smape 16.45
smape 25.02
smape 18.56
smape 26.12
smape 20.78
smape 22.59
smape 77.33
smape 16.23
smape 19.28
smape 24.61
smape 28.79
smape 45.62
smape 45.32
smape 49.64
smape 32.62
smape 29.62
smape 35.05
smape 17.15
smape 19.34
smape 48.98
smape 16.72
smape 38.43
smape 30.61
smape 16.03
smape 26.10
smape 30.76
smape 19.97
smape 39.91
smape 30.08
smape 19.59
smape 21.34
smape 34.33
smape 39.17
smape 35.24
smape 29.38
smape 49.18
smape 48.24
smape 37.51
smape 15.66
smape 32.29
smape 24.44
smape 21.82
smape 13.20
smape 14.14
smape 17.17
smape 16.16
smape 26.28
smape 43.24
smape 38.17
smape 16.82
smape 16.47
smape 47.98
smape 48.08
smape 35.74
smape 33.80
smape 38.26
smape 33.55
smape 30.82
smape 58.78
smape 41.48
smape 34.23
smape 21.23
smape 51.91
smape 28.03
smape 37.61
smape 25.14
smape 26.40
smape 84.73
smape 83.40
smape 28.67
smape 76.85
smape 55.63
smape 89.11
smap

In [6]:
res = np.zeros(len(test['Num']))
for i in range(len(test['Num'])):
    item = test['item_id'][i]
    k = len(X[item])
    naive = last[item][-1]
    if model[item] is not None:
        res[i] = model[item].predict([X_test[i]])[0]
    else:
        res[i] = naive
        
    if smape(np.array([res[i]]), np.array([naive])) > 10:
        res[i] = naive
        
    if res[i] != naive:
        print("corrected", res[i], last[item][-1])

corrected 31157.6386719 33182
corrected 29862.578125 32835
corrected 489852.5625 520754
corrected 23977.875 21847
corrected 11806.3994141 12004
corrected 87510.109375 91130
corrected 31607.5820312 30610
corrected 20038.90625 21404
corrected 43912.4101562 47953
corrected 15637.0976562 16571
corrected 181659.78125 199393
corrected 18683.8222656 18225
corrected 15751.2158203 16624
corrected 8712.32519531 8077
corrected 2205155.0 2205249
corrected 610169.5 597830
corrected 121643.164062 120646
corrected 14301.4042969 14435
corrected 29812.8789062 29680
corrected 382095.375 411858
corrected 248476.03125 253328
corrected 93819.1171875 91670
corrected 53877.5078125 57813
corrected 17633.8007812 17626
corrected 285548.15625 286472
corrected 787505.125 758957
corrected 32526.1289062 30549
corrected 102294.953125 94527
corrected 88410.8203125 89020
corrected 184750.796875 173208
corrected 113969.585938 111239
corrected 246826.140625 250158
corrected 384924.625 370076
corrected 87601.6796875 8572

In [7]:
sample_submission = pd.read_csv("sample_submission.tsv")
sample_submission['y'] = res

In [8]:
sample_submission.to_csv("submission3003.tsv", index=False)